<a href="https://colab.research.google.com/github/FT-Agreed/11785-Project/blob/main/Agreed_tool_daily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Run this cell to start Installing and Authenticate
# Installs geemap package
import subprocess

try:
        import geemap 
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
        print('geopandas package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
        print('tsmoothie package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tsmoothie'])
        print('plotly.express package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'plotly.express'])
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tornado==5.1.0'])
        

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from tsmoothie.smoother import LowessSmoother
import plotly.express as px
import plotly.graph_objects as go
from datetime import date
import ee
import geemap
import requests

try:
        from google.colab import files
        from google.colab import drive
        IN_COLAB = True
except:
        IN_COLAB = False



geemap package not installed. Installing ...
geopandas package not installed. Installing ...
tsmoothie package not installed. Installing ...
plotly.express package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=l9r432K3PYnfGVuFJwBOTZmvSFzvjePMgIbmi05ofjs&tc=rLicO0ONjaIL5o45_5OjIg7M-DUiA0xLubKt9w7iNt0&cc=CYKCItWhwB71iecA5dPZXAIXl8qlU_cvTsya2AUR07w

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qjrlKnWb9FT9gkyhe8HPtHoeDbiKq_dJHaakAs63nEYkfiqxwCM5LE

Successfully saved authorization token.


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#@title Run this cell to select GeoJSON file

user = "Fernigithub"
repo = "files"
gdf = None
url = f"https://api.github.com/repos/{user}/{repo}/git/trees/main?recursive=1"
r = requests.get(url)
res = r.json()

file_list=[]
for file in res["tree"]:
    file_list.append(file["path"])
    
checkitems = [widgets.Checkbox(value=False, description=label.split('.')[0]) for label in file_list]

def sel_f (b):
    global gdf
    selected_data = []
    for j,item in enumerate(checkitems):
        if item.value == True:
            selected_data.append(item.description)

    links = [str(f'https://raw.githubusercontent.com/Fernigithub/files/main/{x}.geojson') for x in selected_data]

    gdf = gpd.GeoDataFrame()
    for item in links:
      gdf = gdf.append(gpd.read_file(item))

for item in checkitems:
    item.observe(sel_f,names='value')
    
widgets.GridBox(checkitems, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

GridBox(children=(Checkbox(value=False, description='Frog_selsey_outwoods'), Checkbox(value=False, description…

In [3]:
#@title Run this cell to start or Reset the selection
# Functions

def smooth(serie,smooth_frac=0.02,itera=3):
    serieResult = pd.Series(serie.values)
    serieindex = pd.Series(serie.index)
    smoother = LowessSmoother(smooth_fraction=smooth_frac, iterations=itera)
    dataSmooth = smoother.smooth(serieResult.values)
    return pd.Series(dataSmooth.smooth_data[0],index=serieindex)
    
def hampel_filter_pandas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, center=True).mean()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median) * -1 
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def hampel_filter_pandas_positivas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, win_type = 'gaussian', center=True).mean(std=1)
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median)
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def cloud (image): 
    image = ee.Image(image)
    clouds = ee.Image(ee.List(image.get('cloud_mask'))).select('probability')
    clean = clouds.lte(80)
    qa = image.select('QA60')
    scl = image.select('SCL').eq(4).Or(image.select('SCL').eq(5))
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    finalmask = mask.And(clean)
    return image.updateMask(finalmask)

imageCollection = ee.ImageCollection("COPERNICUS/S2")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask') \
                                    .apply(**{'primary': imageCollection,'secondary': s2Clouds, \
                                    'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})})



# Aggregation
def aggreg (image_collection, startdate : str ,enddate : str ,timeunit : str ,aggregation : int )-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string
    aggregation: int
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

# Aggregation
def aggreg_sum (image_collection, startdate : str ,enddate : str ,timeunit : str, aggregation : int )-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string
    aggregation: int
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    imag_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(imag_list).map(check, True)

def SM_data (table):
    images = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture') \
    .filterDate('2015-06-01',str(date.today())) \
    .filterBounds(table) \
    .select(['susm'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)

    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).multiply(1).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))

    img_list = ee.List.sequence(0, timedelta.int(),aggregation).map(step).map(filter)

    # Check missing values
    def check(image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image

    return ee.ImageCollection(img_list).map(check, True)

def GPM_data (table):
    images = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filterDate('2022-01-01',str(date.today())) \
    .filterBounds(table) \
    .select(['precipitationCal'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

def kendall_calc(collection):
  afterFilter = ee.Filter.lessThan(**{'leftField': 'system:time_start','rightField': 'system:time_start'})
  joined = ee.ImageCollection(ee.Join.saveAll('after').apply(**{'primary': collection,'secondary': collection,'condition': afterFilter}))

  def sign(i, j):
    return ee.Image(j).neq(i) \
            .multiply(ee.Image(j).subtract(i).clamp(-1, 1)).int()

  def func_oca(current):
    afterCollection = ee.ImageCollection.fromImages(current.get('after'))
    return afterCollection.map(lambda x : ee.Image(sign(current, x)).unmask(0))

  return ee.ImageCollection(joined.map(func_oca).flatten()).reduce('sum', 2)

def selectall(b):
  farm = b['owner'].description
  data = gdf.loc[gdf['Farm'] == farm].Field.values
  if b.old ==False:
    for check_group in checkbox_list:
      for item in check_group:
        if item.description in data:
          item.value = True
  else:
    for check_group in checkbox_list:
      for item in check_group:
        if item.description in data:
          item.value = False

if gdf is not None:
    gdf.crs = "epsg:4326"
    table_json = gdf.__geo_interface__
    table = ee.FeatureCollection(table_json['features'])
    coll = SM_data(table)
    # kendall = kendall_calc(coll).resample('bicubic').reproject('EPSG:4326',None,5000)
    kendall = kendall_calc(coll)
    # minmax = kendall.reduceRegion(ee.Reducer.minMax(),table ,1000).getInfo()
    # print(minmax)
    GPM_coll = GPM_data(coll)
    palette = ['red', 'white', 'green']

    selected_fields = ee.FeatureCollection([])
    Map = geemap.Map()
    farms = ee.ImageCollection("projects/ee-fernandotentor/assets/Agreed/Farms")
    Map.add_basemap('SATELLITE')
    Map.addLayer(kendall,{'min':-150,'max':150,'palette':palette}, 'Soil M. Trend',False)
    Map.addLayer(GPM_coll,{'min':0,'max':150,'palette':palette}, 'GPM_coll',False)
    Map.addLayer(farms, {}, 'Farms_names',0)
    Map.addLayer(table, {}, 'Farm')
    Map.centerObject(table, 10)
    Map.add_labels(table,'Field',font_color='yellow',font_size='10',draggable=False)
    display(Map)

    farms = gdf.Farm.unique()
    n = 6 #items by line
    checkbox_list = []
    for farm in farms:
        data = gdf.loc[gdf['Farm'] == farm].Field
        checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
        checkbox_list.append(checkboxes)

    for j,check_group in enumerate(checkbox_list):
        i=0
        checkall = widgets.Checkbox(value=False, description=farms[j])
        display(farms[j],checkall)
        checkall.observe(selectall,names='value')
        for item in range(0,len(check_group),n ):
            output = widgets.HBox(children=check_group[i:i+n])
            i += n
            display(output)

    time_start = '2017-01-01'
    time_end =  '2022-07-31'
else:
    print('Please select a file')


Map(center=[52.61003816061699, -0.8290340885779912], controls=(WidgetControl(options=['position', 'transparent…

'Loddington'

Checkbox(value=False, description='Loddington')

In [4]:
#@title Select Time Ranges
time_start = '2017-01-01' #@param {type:"date"}
time_end =  '2022-07-31' #@param {type:"date"}
aggregation_unit = 'day' #@param ["day", "month", "year"]
aggregation = 1 #@param {type:"number"}
print('Selected Range is from ' + time_start + ' to ' + time_end)
print('Aggregation ' + str(aggregation) +' '+ aggregation_unit)

Selected Range is from 2017-01-01 to 2022-07-31
Aggregation 1 day


In [5]:
#@title Run this cell to elect the above checked fields
selected_data = []
selected = []
for j,check_group in enumerate(checkbox_list):
  for i in range(0, len(check_group)):
      if check_group[i].value == True:
          selected_data = selected_data + [check_group[i].description]

print(selected_data)
selected = table.filter(ee.Filter.inList("Field",selected_data))
selected_fields = selected_fields.merge(selected).distinct('Field')
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
print('Fields selected',len(selected_fields.getInfo()['features']))

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
Fields selected 12


# Select fields GUI tool

In [6]:
#@title Run this cell to select the previously selected field by GUI 
roi = ee.FeatureCollection(Map.draw_features)
selected = table.filterBounds(roi.union(10))
print(selected.size().getInfo(),'features selected by drawing tool')
selected_fields = selected_fields.merge(selected).distinct('Field')
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
try:
  Map.remove_drawn_features()
except:
  None
print('Fields selected', selected_fields.aggregate_array('Field').getInfo())



0 features selected by drawing tool
Fields selected ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


# Add other geometries [Optional]

In [7]:
#@title Run this cell after drawing new geometries at GUI
featcoll = []
i = 0
for feat in Map.draw_features:
    i +=1
    feat = feat.set({'properties':{'Farm':'N/A','Field':'custom '+str(i)}})
    featcoll.append(feat)
print(len(featcoll),'features added')
added_geom = ee.FeatureCollection(featcoll)
try:
  selected_fields = selected_fields.merge(added_geom).distinct('Field')
except:
  selected_fields = added_geom
  Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
print('Fields selected', selected_fields.aggregate_array('Field').getInfo())
Map.addLayer(selected_fields, {'color':'yellow'}, "Selected fields")
Map.draw_features.clear()

try:
  Map.add_labels(selected_fields,'Field',font_color='yellow',font_size='10')
  Map.remove_drawn_features()
except:
  None


0 features added
Fields selected ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']


# Save selected geometries

In [ ]:
#@title Run this cell to save the selected geometries as GeoJSON.
# Save geometries as GeoJson 
geojson_feat = selected_fields.getInfo()
with open('selected_features.geojson', 'w') as f:
    json.dump(geojson_feat, f)
if IN_COLAB:
    files.download('selected_features.geojson')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
#@title Run this cell to get Crop ID from CROME
# Crop Map of England
# Rasterization LUC code into integer with Qgis
dic = {'AC01': 81,'AC03': 1, 'AC04': 2, 'AC05': 3, 'AC06': 4, 'AC07': 5, 'AC09': 6, 'AC10': 7, 'AC14': 8, 'AC15': 9, 'AC16': 10, 'AC17': 11, 'AC18': 12, 'AC19': 13, 'AC20': 14, 'AC22': 15, 'AC23': 16, 'AC24': 17, 'AC26': 18, 'AC27': 19, 'AC30': 20, 'AC32': 21, 'AC34': 22, 'AC35': 23, 'AC36': 24, 'AC37': 25, 'AC38': 26, 'AC41': 27, 'AC44': 28, 'AC45': 29, 'AC50': 30, 'AC52': 31, 'AC58': 32, 'AC59': 33, 'AC60': 34, 'AC61': 35, 'AC62': 36, 'AC63': 37, 'AC64': 38, 'AC65': 39, 'AC66': 40, 'AC67': 41, 'AC68': 42, 'AC69': 43, 'AC70': 44, 'AC71': 45, 'AC72': 46, 'AC74': 47, 'AC81': 48, 'AC88': 49, 'AC90': 50, 'AC92': 51, 'AC94': 52, 'AC100': 53, 'CA02': 54, 'LG01': 55, 'LG02': 56, 'LG03': 57, 'LG04': 58, 'LG06': 59, 'LG07': 60, 'LG09': 61, 'LG08': 62, 'LG11': 63, 'LG13': 64, 'LG14': 65, 'LG15': 66, 'LG16': 67, 'LG20': 68, 'LG21': 69, 'SR01': 70, 'FA01': 71, 'HE02': 72, 'HEAT': 73, 'PG01': 74, 'NA01': 75, 'WA00': 76, 'TC01': 77, 'NU01': 78, 'WO12': 79, 'AC00': 80,'WA01': 80}
luc_dict = dict(zip(dic.values(), dic.keys()))

# CROME LUC CODE
luc_code = ['AC01','AC03','AC04','AC05','AC06','AC07','AC09','AC10','AC14','AC15','AC16','AC17','AC18','AC19','AC20','AC22','AC23','AC24','AC26','AC27','AC30','AC32','AC34','AC35','AC36','AC37','AC38','AC41','AC44','AC45','AC50','AC52','AC58','AC59','AC60','AC61','AC62','AC63','AC64','AC65','AC66','AC67','AC68','AC69','AC70','AC71','AC72','AC74','AC81','AC88','AC90','AC92','AC94','AC100','CA02','LG01','LG02','LG03','LG04','LG06','LG07','LG09','LG08','LG11','LG13','LG14','LG15','LG16','LG20','LG21','SR01','FA01','HE02','HEAT','PG01','NA01','WA00','TC01','NU01','WO12','AC00','WA01']
crop = ['Spring_Barley', 'Beet', 'Borage', 'Buckwheat', 'Canary_Seed', 'Carrot', 'Chicory', 'Daffodil', 'Hemp', 'Lettuce', 'Spring_Linseed_', 'Maize', 'Millet', 'Spring_Oats', 'Onions', 'Parsley', 'Parsnips', 'Spring_Rye', 'Spinach', 'Strawberry', 'Spring_Triticale', 'Spring_Wheat', 'Spring_Cabbage', 'Turnip', 'Spring_Oilseed', 'Brown_Mustard', 'Mustard', 'Radish', 'Potato', 'Tomato', 'Squash', 'Siam_Pumpkin', 'Mixed_Crop_Group_1', 'Mixed_Crop_Group_2', 'Mixed_Crop_Group_3', 'Mixed_Crop_Group_4', 'Mixed_Crop_Group_5', 'Winter_Barley', 'Winter_Linseed', 'Winter_Oats', 'Winter_Wheat', 'Winter_Oilseed', 'Winter_Rye', 'Winter_Triticale', 'Winter_Cabbage', 'Coriander', 'Corn_gromwell', 'Phacelia', 'Poppy', 'Sunflower', 'Gladioli', 'Sorghum', 'Sweet_William', 'Italian_Ryegrass', 'Cover_Crop', 'Chickpea', 'Fenugreek', 'Spring_Field_beans', 'Green_Beans', 'Lupins', 'Spring_Peas', 'Cowpea', 'Soya', 'Lucerne', 'Sainfoin', 'Clover', 'Mixed_Crops–Group_1_Leguminous', 'Mixed_Crops–Group_2_Leguminous', 'Winter_Field_beans', 'Winter_Peas', 'Short_Rotation_Coppice', 'Fallow_Land', 'Heathland_and_Bracken', 'Heather', 'Grass', 'Non_vegetated_or_sparsely_vegetated_Land', 'Water', 'Perennial_Crops_and_Isolated_Trees', 'Nursery_Crops', 'Trees_and_Scrubs,_short_Woody_plants,_hedgerows', 'Unknown_or_Mixed_Vegetation', 'Unknown_or_Mixed_Vegetation']
crop_dict = dict(zip(luc_code,crop))

crome2018 = ee.Image("users/fertentor/Crop_Map_of_England_2018").select(['b1'],['2018'])
crome2017 = ee.Image("users/fertentor/Crop_Map_of_England_2017").select(['b1'],['2017'])
crome2019 = ee.Image("users/fertentor/Crop_Map_of_England_2019").select(['b1'],['2019'])
crome2020 = ee.Image("users/fertentor/Crop_Map_of_England_2020").select(['b1'],['2020'])
crome2021 = ee.Image("users/fertentor/Crop_Map_of_England_2021").select(['b1'],['2021'])
crome2016 = ee.Image("users/fertentor/Crop_Map_of_England_2016").select(['b1'],['2016'])

ImgColl = ee.ImageCollection.fromImages([crome2016,crome2017,crome2018,crome2019,crome2020,crome2021])

def redu (img):
    bandname = ee.List(img.bandNames()).get(0)
    feat = img.reduceRegions(selected_fields,ee.Reducer.mode(),10)
    feat = feat.map(lambda f: f.set({'mode':ee.String(ee.Number(f.get('mode')).int())}))
    return feat.map(lambda f : f.set({'year':bandname}))

crop = ImgColl.map(redu).flatten()

df = gpd.GeoDataFrame.from_features(crop.getInfo()['features'])
df['luccode'] = [luc_dict[i] for i in df['mode']]
df['crop'] = [crop_dict[i] for i in df['luccode']]
display(df.loc[:,['Farm','Field','year','luccode','crop']])

df.loc[:,['Farm','Field','year','luccode','crop']].to_csv('CROME.csv')

if IN_COLAB:
    files.download('CROME.csv')
    



,Farm,Field,year,luccode,crop
0,Loddington,1,2016,AC32,Spring_Wheat
1,Loddington,2,2016,PG01,Grass
2,Loddington,3,2016,AC30,Spring_Triticale
3,Loddington,4,2016,AC67,Winter_Oilseed
4,Loddington,5,2016,AC66,Winter_Wheat
...,...,...,...,...,...
67,Loddington,8,2021,PG01,Grass
68,Loddington,9,2021,SR01,Short_Rotation_Coppice
69,Loddington,10,2021,WA01,Unknown_or_Mixed_Vegetation
70,Loddington,11,2021,AC66,Winter_Wheat


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#@title Run this cell to get soil Texture
dic_taxon = {0:'NODATA', 1:'Albaqualfs', 2:'Cryaqualfs', 4:'Durixeralfs', 6:'Endoaqualfs', 7:'Epiaqualfs', 9:'Fragiaqualfs', 10:'Fragiudalfs', 11:'Fragixeralfs', 12:'Fraglossudalfs', 13:'Glossaqualfs', 14:'Glossocryalfs', 15:'Glossudalfs', 16:'Haplocryalfs', 17:'Haploxeralfs', 18:'Hapludalfs', 19:'Haplustalfs', 25:'Natraqualfs', 26:'Natrixeralfs', 27:'Natrudalfs', 28:'Natrustalfs', 29:'Palecryalfs', 30:'Paleudalfs', 31:'Paleustalfs', 32:'Palexeralfs', 38:'Rhodustalfs', 39:'Vermaqualfs', 41:'Eutroboralfs', 42:'Ochraqualfs', 43:'Glossoboralfs', 44:'Cryoboralfs', 45:'Natriboralfs', 46:'Paleboralfs', 50:'Cryaquands', 58:'Fulvicryands', 59:'Fulvudands', 61:'Haplocryands', 63:'Haploxerands', 64:'Hapludands', 74:'Udivitrands', 75:'Ustivitrands', 76:'Vitraquands', 77:'Vitricryands', 80:'Vitrixerands', 82:'Aquicambids', 83:'Aquisalids', 85:'Argidurids', 86:'Argigypsids', 87:'Calciargids', 89:'Calcigypsids', 90:'Gypsiargids', 92:'Haplargids', 93:'Haplocalcids', 94:'Haplocambids', 96:'Haplodurids', 97:'Haplogypsids', 98:'Haplosalids', 99:'Natrargids', 100:'Natridurids', 101:'Natrigypsids', 102:'Paleargids', 103:'Petroargids', 104:'Petrocalcids', 105:'Petrocambids', 107:'Petrogypsids', 110:'Calciorthids', 111:'Camborthids', 112:'Paleorthids', 113:'Durorthids', 114:'Durargids', 115:'Gypsiorthids', 116:'Nadurargids', 118:'Cryaquents', 119:'Cryofluvents', 120:'Cryopsamments', 121:'Cryorthents', 122:'Endoaquents', 123:'Epiaquents', 124:'Fluvaquents', 126:'Frasiwassents', 131:'Hydraquents', 133:'Psammaquents', 134:'Psammowassents', 135:'Quartzipsamments', 136:'Sulfaquents', 137:'Sulfiwassents', 138:'Torrifluvents', 139:'Torriorthents', 140:'Torripsamments', 141:'Udifluvents', 142:'Udipsamments', 143:'Udorthents', 144:'Ustifluvents', 145:'Ustipsamments', 146:'Ustorthents', 147:'Xerofluvents', 148:'Xeropsamments', 149:'Xerorthents', 153:'Udarents', 154:'Torriarents', 155:'Xerarents', 179:'Cryofibrists', 180:'Cryofolists', 181:'Cryohemists', 182:'Cryosaprists', 183:'Frasiwassists', 184:'Haplofibrists', 185:'Haplohemists', 186:'Haplosaprists', 189:'Sphagnofibrists', 190:'Sulfihemists', 191:'Sulfisaprists', 196:'Udifolists', 201:'Borosaprists', 202:'Medisaprists', 203:'Borohemists', 206:'Calcicryepts', 207:'Calciustepts', 208:'Calcixerepts', 209:'Cryaquepts', 210:'Durixerepts', 212:'Durustepts', 213:'Dystrocryepts', 215:'Dystroxerepts', 216:'Dystrudepts', 217:'Dystrustepts', 218:'Endoaquepts', 219:'Epiaquepts', 220:'Eutrudepts', 221:'Fragiaquepts', 222:'Fragiudepts', 225:'Halaquepts', 226:'Haplocryepts', 228:'Haploxerepts', 229:'Haplustepts', 230:'Humaquepts', 231:'Humicryepts', 233:'Humixerepts', 234:'Humudepts', 235:'Humustepts', 245:'Ustochrepts', 246:'Eutrochrepts', 247:'Dystrochrepts', 248:'Eutrocryepts', 249:'Haplaquepts', 250:'Xerochrepts', 251:'Cryochrepts', 252:'Fragiochrepts', 253:'Haplumbrepts', 254:'Cryumbrepts', 255:'Dystropepts', 256:'Vitrandepts', 268:'Argialbolls', 269:'Argiaquolls', 270:'Argicryolls', 271:'Argiudolls', 272:'Argiustolls', 273:'Argixerolls', 274:'Calciaquolls', 275:'Calcicryolls', 276:'Calciudolls', 277:'Calciustolls', 278:'Calcixerolls', 279:'Cryaquolls', 280:'Cryrendolls', 283:'Durixerolls', 284:'Durustolls', 285:'Endoaquolls', 286:'Epiaquolls', 287:'Haplocryolls', 289:'Haploxerolls', 290:'Hapludolls', 291:'Haplustolls', 292:'Haprendolls', 294:'Natraquolls', 296:'Natrixerolls', 297:'Natrudolls', 298:'Natrustolls', 299:'Palecryolls', 300:'Paleudolls', 301:'Paleustolls', 302:'Palexerolls', 303:'Vermudolls', 306:'Haploborolls', 307:'Argiborolls', 308:'Haplaquolls', 309:'Cryoborolls', 310:'Natriborolls', 311:'Calciborolls', 312:'Paleborolls', 342:'Alaquods', 343:'Alorthods', 345:'Duraquods', 348:'Durorthods', 349:'Endoaquods', 350:'Epiaquods', 351:'Fragiaquods', 353:'Fragiorthods', 354:'Haplocryods', 356:'Haplohumods', 357:'Haplorthods', 358:'Humicryods', 367:'Haplaquods', 368:'Cryorthods', 370:'Albaquults', 371:'Endoaquults', 372:'Epiaquults', 373:'Fragiaquults', 374:'Fragiudults', 375:'Haplohumults', 376:'Haploxerults', 377:'Hapludults', 378:'Haplustults', 381:'Kandiudults', 385:'Kanhapludults', 387:'Paleaquults', 388:'Palehumults', 389:'Paleudults', 390:'Paleustults', 391:'Palexerults', 396:'Rhodudults', 399:'Umbraquults', 401:'Ochraquults', 403:'Calciaquerts', 405:'Calciusterts', 406:'Calcixererts', 409:'Dystraquerts', 410:'Dystruderts', 412:'Endoaquerts', 413:'Epiaquerts', 414:'Gypsitorrerts', 415:'Gypsiusterts', 417:'Haplotorrerts', 418:'Haploxererts', 419:'Hapluderts', 420:'Haplusterts', 422:'Natraquerts', 424:'Salitorrerts', 429:'Chromusterts', 430:'Pellusterts', 431:'Chromoxererts', 432:'Pelluderts', 433:'Torrerts'}
dic_tex = {1:'Cl', 2:'SiCl', 3:'SaCl', 4:'ClLo', 5:'SiClLo', 6:'SaClLo', 7:'Lo', 8:'SiLo', 9:'SaLo', 10:'Si', 11:'LoSa', 12:'Sa'}

# Soil Texture
soils = ee.Image("OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01")

#Soil Texture Class
soil_texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

#Soil PH
ph = ee.Image("OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02")

#Soil Organic Carbon
socc = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02")

#Soil Bulk Density
bd = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02")

ImgColl = ee.ImageCollection.fromImages([soils,soil_texture,ph,socc,bd])

img_ids = [
  "OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01",
  "OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02",
  "OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02",
  "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02",
  "OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02"]

img_values = ['TAXONOMY','TEXTURE','PH','SOC','BD']

dic_values = dict(zip(img_ids,img_values))

def redu (img):
    bandname = ee.List(img.bandNames()).get(0)
    feat = img.reduceRegions(selected_fields,ee.Reducer.mode(),10)
    # feat = feat.map(lambda f: f.set({'mode':ee.String(ee.Number(f.get('mode')).int())}))
    img_name = img.get('system:id')
    return feat.map(lambda f : f.set({'bandname':bandname,'img_name':img_name}))

crop = ImgColl.map(redu).flatten()

df = gpd.GeoDataFrame.from_features(crop.getInfo()['features'])
df['type'] = [dic_values[i] for i in df['img_name']]

df2 = gpd.GeoDataFrame()
for i, group in df.groupby('type'):
    if i == 'TAXONOMY':
        group['value'] = [ dic_taxon[int(x)] if not np.isnan(x) else 'x' for x in group['mode']]
        df2 = df2.append(group)
    elif i == 'TEXTURE':
        group['b0'] = [ dic_tex[int(x)] for x in group['b0']]
        group['b10'] = [ dic_tex[int(x)] for x in group['b10']]
        group['b30'] = [ dic_tex[int(x)] for x in group['b30']]
        group['b60'] = [ dic_tex[int(x)] for x in group['b60']]
        group['b100'] = [ dic_tex[int(x)] for x in group['b100']]
        group['b200'] = [ dic_tex[int(x)] for x in group['b200']]
        df2 = df2.append(group)
    elif i == 'BD':
        group['b0'] = [ str(int(x)) for x in group['b0']]
        group['b10'] = [ str(int(x)) for x in group['b10']]
        group['b30'] = [ str(int(x)) for x in group['b30']]
        group['b60'] = [ str(int(x)) for x in group['b60']]
        group['b100'] = [ str(int(x)) for x in group['b100']]
        group['b200'] = [ str(int(x)) for x in group['b200']]
        df2 = df2.append(group)
    elif i == 'PH':
        group['b0'] = [ int(x) / 10 for x in group['b0']]
        group['b10'] = [ int(x) / 10 for x in group['b10']]
        group['b30'] = [ int(x) / 10 for x in group['b30']]
        group['b60'] = [ int(x) / 10 for x in group['b60']]
        group['b100'] = [ int(x) / 10 for x in group['b100']]
        group['b200'] = [ int(x) / 10 for x in group['b200']]
        df2 = df2.append(group)
    elif i == 'SOC':
        group['b0'] = [ int(x) for x in group['b0']]
        group['b10'] = [ int(x) for x in group['b10']]
        group['b30'] = [ int(x) for x in group['b30']]
        group['b60'] = [ int(x) for x in group['b60']]
        group['b100'] = [ int(x) for x in group['b100']]
        group['b200'] = [ int(x) for x in group['b200']]
        df2 = df2.append(group)
    
display(df2.loc[:,['Farm','Field','type','value','b0','b10','b30','b60','b100','b200']])
df2.loc[:,['Farm','Field','type','value','b0','b10','b30','b60','b100','b200']].to_csv('soil.csv',index=False)

if IN_COLAB:
    files.download('soil.csv')

,Farm,Field,type,value,b0,b10,b30,b60,b100,b200
48,Loddington,1,BD,NaN,126,128,143,158,162,165
49,Loddington,2,BD,NaN,118,125,140,156,159,162
50,Loddington,3,BD,NaN,131,134,146,156,156,158
51,Loddington,4,BD,NaN,130,131,144,154,161,162
52,Loddington,5,BD,NaN,126,129,139,150,150,155
53,Loddington,6,BD,NaN,126,130,143,159,160,165
54,Loddington,7,BD,NaN,126,128,141,150,157,159
55,Loddington,8,BD,NaN,124,129,142,157,161,165
56,Loddington,9,BD,NaN,128,129,145,156,160,160
57,Loddington,10,BD,NaN,129,134,145,150,153,157


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# NDMI

In [ ]:
#@title Run this cell to create NDMI Chart
#### obtain NDMI data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    ndmi = image.normalizedDifference(['B8','B11']).rename('ndmi')
    data = ndmi.reduceRegions(selected_fields ,ee.Reducer.mean(),10).filterMetadata('mean','greater_than',0)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    ndmi_value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,ndmi_value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'ndmi_mean',ndmi_value,'date',dia_list,'crop_id',crop_id)

images = s2SrWithCloudMask.filterBounds(selected_fields.geometry(10)).map(cloud)
S2_list_clean = aggreg(images ,time_start ,time_end,aggregation_unit,aggregation) #'day' or 'month'
image_list = ee.List(S2_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(S2_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    ndmi_mean = feat['properties']['ndmi_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = ndmi_mean

df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel[column] = hampel_filter_pandas(df_int[column],1,0.5)
    df_hampel[column] = hampel_filter_pandas_positivas(df_hampel[column],2,2)
    df_hampel_smooth[column] = smooth(df_hampel[column],0.02,3)
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

# Plot NDMI 
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_plot['date'] = df_plot.index.strftime("%Y-%m-%d")
df_ndmi = df_plot
fig = px.line(df_plot, x=df_plot.date, y=df_plot.columns , range_y=[0,1],
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "NDMI value",
                     "index": "Date",
                     "variable": "Fields"
                 },
                title="NDMI by Fields")
                
color_list = px.colors.qualitative.Light24 * 3
for i,item in enumerate(fied_names):
    fig.add_trace(go.Scatter(mode='markers', x= df.index.strftime("%Y-%m-%d") , y = df.loc[:,item] , 
         name = 'raw '+item,
        marker_size=5 , marker_color=color_list[i] ,visible = "legendonly"))

fig.update_xaxes(
    # tickangle=45,
    tickfont=dict(size=10),
    dtick="M1",
    tickformat="%b\n\n\n%Y",
    ticklabelmode="period")

# edit text and hovertemplate
for ser in fig['data']:
    ser['text']=list(set([d for d in df_plot['date']]))
    ser['hovertemplate']='Field='+ser['name']+'<br>date=%{text}<br>value=%{y}<extra></extra>'

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show(figsize=(20,10))


## Save data

In [ ]:
#@title Run this cell to save NDMI as .CSV file
df_ndmi.to_csv('NDMI_data.csv')
df_ndmi.to_excel('NDMI_data.xls')

# NDVI

In [ ]:
#@title Run this cell to create NDVI Chart
#### obtain NDVI data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi')
    data = ndvi.reduceRegions(selected_fields ,ee.Reducer.mean(),10).filterMetadata('mean','greater_than',0)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    ndvi_value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,ndvi_value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'ndvi_mean',ndvi_value,'date',dia_list,'crop_id',crop_id)

images = s2SrWithCloudMask.filterBounds(selected_fields.geometry(10)).map(cloud)
S2_list_clean = aggreg(images ,time_start ,time_end ,aggregation_unit,aggregation) #'day' or 'month'
image_list = ee.List(S2_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(S2_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    ndvi_mean = feat['properties']['ndvi_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = ndvi_mean

df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel[column] = hampel_filter_pandas(df_int[column],1,0.5)
    df_hampel[column] = hampel_filter_pandas_positivas(df_hampel[column],2,2)
    df_hampel_smooth[column] = smooth(df_hampel[column],0.02,3)
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

# Plot NDMI 
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_plot['date'] = df_plot.index.strftime("%Y-%m-%d")
df_ndvi = df_plot
fig = px.line(df_plot, x=df_plot.date, y=df_plot.columns , range_y=[0,1],
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "NDVI value",
                     "index": "Date",
                     "variable": "Fields"
                 },
                title="NDVI by Fields")
                
color_list = px.colors.qualitative.Light24 * 3
for i,item in enumerate(fied_names):
    fig.add_trace(go.Scatter(mode='markers', x= df.index.strftime("%Y-%m-%d") , y = df.loc[:,item] , 
         name = 'raw '+item,
        marker_size=5 , marker_color=color_list[i] ,visible = "legendonly"))

fig.update_xaxes(
    # tickangle=45,
    tickfont=dict(size=10),
    dtick="M1",
    tickformat="%b\n\n\n%Y",
    ticklabelmode="period")

# edit text and hovertemplate
for ser in fig['data']:
    ser['text']=list(set([d for d in df_plot['date']]))
    ser['hovertemplate']='Field='+ser['name']+'<br>date=%{text}<br>value=%{y}<extra></extra>'

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show(figsize=(20,10))

In [ ]:
#@title Run this cell to save NDVI data as .CSV
df_ndvi.to_csv('NDVI_data.csv')
df_ndvi.to_excel('NDVI_data.xls')

# NOx

In [ ]:
#@title Run this cell to create NOx Chart
#### obtain NOx data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),100)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'nox_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').select(['NO2_column_number_density'],['NO2'])
NOx_list_clean = aggreg(images ,time_start ,time_end ,aggregation_unit,aggregation) #'day' or 'month'
image_list = ee.List(NOx_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(NOx_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    nox_mean = feat['properties']['nox_mean']
    if fechas.size != 0 :
        try:
            df.loc[fechas,field_name] = nox_mean
        except:
            pass



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel_smooth[column] = smooth(df_int[column],0.02,3)

df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

# Plot NOx
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_plot['date'] = df_plot.index.strftime("%Y-%m-%d")
df_nox = df_plot
fig = px.line(df_plot, x=df_plot.date, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "NOx value",
                     "index": "Date",
                     "variable": "Fields"
                 },
                title="NOx by Fields")
                
color_list = px.colors.qualitative.Light24 * 3
for i,item in enumerate(fied_names):
    fig.add_trace(go.Scatter(mode='markers', x= df.index.strftime("%Y-%m-%d") , y = df.loc[:,item] , 
         name = 'raw '+item,
        marker_size=5 , marker_color=color_list[i] ,visible = "legendonly"))

fig.update_xaxes(
    # tickangle=45,
    tickfont=dict(size=10),
    dtick="M1",
    tickformat="%b\n\n\n%Y",
    ticklabelmode="period")

# edit text and hovertemplate
for ser in fig['data']:
    ser['text']=list(set([d for d in df_plot['date']]))
    ser['hovertemplate']='Field='+ser['name']+'<br>date=%{text}<br>value=%{y}<extra></extra>'

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show(figsize=(20,10))

In [ ]:
#@title Run this cell to save NOx as .CSV file
df_nox.to_csv('NOx_data.csv')
df_nox.to_excel('NOx_data.xls')

# Soil Moisture

In [ ]:
#@title Run this cell to create Soil Moisture Chart
#### obtain Soil Moisture data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),100)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'susm_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').select(['susm'],['susm'])
images_list_clean = aggreg(images ,time_start ,time_end,aggregation_unit,aggregation) #'day' or 'month'
image_list = ee.List(images_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(images_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    susm_mean = feat['properties']['susm_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = susm_mean



df_int = df.interpolate( method='time', limit_direction ='both').dropna()

df_hampel = pd.DataFrame()
df_hampel_smooth = pd.DataFrame()
for column in df_int:
    df_hampel_smooth[column] = smooth(df_int[column],0.02,3)
df_hampel_smooth = df_hampel_smooth.interpolate( method='linear', limit_direction ='both').dropna()

# Plot NDMI 
df_plot = df_hampel_smooth
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_plot['date'] = df_plot.index.strftime("%Y-%m-%d")
df_susm = df_plot
fig = px.line(df_plot, x=df_plot.date, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "sussm value",
                     "index": "Date",
                     "variable": "Fields"
                 },
                title="Subsurface soil moisture by Fields")
                
color_list = px.colors.qualitative.Light24 * 3
for i,item in enumerate(fied_names):
    fig.add_trace(go.Scatter(mode='markers', x= df.index.strftime("%Y-%m-%d") , y = df.loc[:,item] , 
         name = 'raw '+item,
        marker_size=5 , marker_color=color_list[i] ,visible = "legendonly"))

fig.update_xaxes(
    # tickangle=45,
    tickfont=dict(size=10),
    dtick="M1",
    tickformat="%b\n\n\n%Y",
    ticklabelmode="period")

# edit text and hovertemplate
for ser in fig['data']:
    ser['text']=list(set([d for d in df_plot['date']]))
    ser['hovertemplate']='Field='+ser['name']+'<br>date=%{text}<br>value=%{y}<extra></extra>'

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show(figsize=(20,10))

In [ ]:
#@title Run this cell to save Sup Soil Moisture as .CSV file
df_susm.to_csv('SuSM_data.csv')
df_susm.to_excel('SuSM_data.xls')

# Precipitation

In [ ]:
#@title Run this cell to create Precipitation Chart
#### obtain Precipitation data

fields = selected_fields.getInfo()


list_fields=[]
for feat in fields['features']:
    field = feat['properties']['Field']
    list_fields.append(field)

list_farms=[]
for feat in fields['features']:
    crop_id = feat['properties']['Farm']
    list_farms.append(crop_id)

list_tuples = list(zip(set(list_fields),list_farms))

def data_extr (image):
    dia = ee.Date(image.get('system:time_start')).format('yyyy-MM-dd')
    data = image.reduceRegions(selected_fields ,ee.Reducer.mean(),1000)
    field_name = data.aggregate_array('Field')
    farm_name = data.aggregate_array('Farm')
    crop_id = 'N/A' #data.aggregate_array('crop_id')
    value = ee.List(data.aggregate_array('mean'))
    dia_list = ee.List.repeat(dia,value.length())
    return ee.Feature(None).set('field',field_name,'farm',farm_name,'gpm_mean',value,'date',dia_list,'crop_id',crop_id)

images = ee.ImageCollection("NASA/GPM_L3/IMERG_V06").select(['precipitationCal'],['precipitationCal'])
images_list_clean = aggreg_sum(images ,time_start ,time_end,aggregation_unit,aggregation) #'day' or 'month'
image_list = ee.List(images_list_clean.aggregate_array('system:time_start')).map(lambda date : ee.Date(date).format('yyyy-MM-dd')).getInfo()
list_dates = list(dict.fromkeys(image_list))
data = ee.ImageCollection(images_list_clean).map(data_extr).getInfo()

x  = pd.DatetimeIndex(list_dates).sort_values()
df = pd.DataFrame(columns = set(list_fields), index = x).astype(float) 
for feat in data['features']:
    field_name = pd.Series(feat['properties']['field'])
    fechas = pd.Series(feat['properties']['date'],dtype='datetime64[ns]')
    gpm_mean = feat['properties']['gpm_mean']
    if fechas.size != 0 :
        df.loc[fechas,field_name] = gpm_mean

df_int = df.interpolate( method='time', limit_direction ='both').dropna()

# Plot NDMI 
df_plot = df_int
fied_names = df_plot.columns.get_level_values(0)
df_plot.columns = fied_names
df_plot['date'] = df_plot.index.strftime("%Y-%m-%d")
df_prec = df_plot
fig = px.line(df_plot, x=df_plot.date, y=df_plot.columns ,
                    color_discrete_sequence=px.colors.qualitative.Light24,  
                    labels={
                     "value": "Precipitation value",
                     "index": "Precipitation",
                     "variable": "Fields"
                 },
                title="Precipitation by Fields")
                
color_list = px.colors.qualitative.Light24 * 3
for i,item in enumerate(fied_names):
    fig.add_trace(go.Scatter(mode='markers', x= df.index.strftime("%Y-%m-%d") , y = df.loc[:,item] , 
         name = 'raw '+item,
        marker_size=5 , marker_color=color_list[i] ,visible = "legendonly"))

fig.update_xaxes(
    # tickangle=45,
    tickfont=dict(size=10),
    dtick="M1",
    tickformat="%b\n\n\n%Y",
    ticklabelmode="period")

# edit text and hovertemplate
for ser in fig['data']:
    ser['text']=list(set([d for d in df_plot['date']]))
    ser['hovertemplate']='Field='+ser['name']+'<br>date=%{text}<br>value=%{y}<extra></extra>'

fig.update_layout(dict(updatemenus=[
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=["visible", "legendonly"],
                                    label="Deselect All",
                                    method="restyle"
                                ),
                                dict(
                                    args=["visible", True],
                                    label="Select All",
                                    method="restyle"
                                )
                            ]),
                            pad={"r": 10, "t": 10},
                            showactive=False,
                            x=1,
                            xanchor="right",
                            y=1.1,
                            yanchor="top"
                        ),
                    ]
              ))
fig.show(figsize=(20,10))

In [ ]:
#@title Run this cell to save Precipitation as .CSV file
df_prec.to_csv('Prec_data.csv')
df_prec.to_excel('Prec_data.xls')

In [ ]:
#@title
